# ACCRE Project - Data Analysis

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.animation as animation

In [ ]:
# Read in fullsample_cleaned.csv as a dataframe
jobs = pd.read_csv("../data/fullsample_cleaned.csv")

# Read in fullsample_cleaned.csv as a dataframe
logs = pd.read_csv("../data/logs.csv")

In [ ]:
# Convert BEGIN and END columns in jobs df to datetime type
jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'])
jobs['END'] = pd.to_datetime(jobs['END'])

# Convert REQTIME, USEDTIME, JOBLENGTH columns in jobs df to timedelta type
jobs['REQTIME'] = pd.to_timedelta(jobs['REQTIME'])
jobs['USEDTIME'] = pd.to_timedelta(jobs['USEDTIME'])
jobs['JOBLENGTH'] = pd.to_timedelta(jobs['JOBLENGTH'])

# Convert DATE column in logs df to datetime type
logs['DATE'] = pd.to_datetime(logs['DATE'])

In [ ]:
# Count the amount of jobs per hour and create new column
jobs['GROUP1H'] = jobs.groupby(pd.Grouper(key='END',freq='H')).transform('count')['JOBID'].sort_index()

## Question 1: Calculate some descriptive statistics for how many jobs per hour are being completed.

In [ ]:
# Total jobs ended each month
weekly_jobs_ended_by_hour = jobs.groupby(pd.Grouper(key='END',freq='M')).count()/168
weekly_jobs_ended_by_hour = weekly_jobs_ended_by_hour.reset_index()

# Total jobs completed each month
weekly_jobs_completed_by_hour = jobs[jobs['STATE'] == 'COMPLETED'].groupby(pd.Grouper(key='END',freq='M')).count()/168
weekly_jobs_completed_by_hour = weekly_jobs_completed_by_hour.reset_index()

# Format date ticks for x axis and define fontsize
date_labels = weekly_jobs_ended_by_hour['END'].dt.strftime('%Y-%m')
fontsize = 20

# Set figure size and graph plots
fig, ax = plt.subplots(figsize=(30,16))
sns.barplot(x = 'END', 
            y = 'JOBID', 
            data = weekly_jobs_ended_by_hour, 
            hue = date_labels, 
            dodge = False, 
            palette = "deep",
            ax = ax,
            alpha = 0.7)
sns.barplot(x = 'END', 
            y = 'JOBID', 
            data = weekly_jobs_completed_by_hour, 
            hue = date_labels, 
            dodge = False, 
            palette = "deep",
            ax = ax)

plt.xlabel('Week', fontsize = fontsize)
plt.xticks(fontsize = fontsize - 2)
ax.set_xticklabels(labels = date_labels)

plt.ylabel('Jobs Per Hour', fontsize = fontsize)
plt.yticks(fontsize = fontsize - 2)

ax.get_legend().set_visible(False);

In [ ]:
jobs

In [ ]:
jobs['END'].describe()

In [ ]:
# Total jobs ended each month
weekly_jobs_ended_median = jobs.groupby(pd.Grouper(key='END',freq='W')).median()
weekly_jobs_ended_median = weekly_jobs_ended_median.reset_index()

# Total jobs completed each month
weekly_jobs_ended_by_hour = jobs.groupby(pd.Grouper(key='END',freq='W')).count()/168
weekly_jobs_ended_by_hour = weekly_jobs_ended_by_hour.reset_index()

# Format date ticks for x axis and define fontsize
date_labels = weekly_jobs_ended_by_hour['END'].dt.strftime('%Y-%m')
fontsize = 20

# Set figure size and graph plots
fig, ax = plt.subplots(figsize=(30,16))
sns.barplot(x = 'END', 
            y = 'GROUP1H', 
            data = weekly_jobs_ended_median, 
            hue = date_labels, 
            dodge = False, 
            palette = "deep",
            ax = ax,
            alpha = 0.7)
sns.barplot(x = 'END', 
            y = 'JOBID', 
            data = weekly_jobs_ended_by_hour, 
            hue = date_labels, 
            dodge = False, 
            palette = "deep",
            ax = ax)

plt.xlabel('Week', fontsize = fontsize)
plt.xticks(fontsize = fontsize - 2, rotation = 45)
ax.set_xticklabels(labels = date_labels)

plt.ylabel('Jobs Per Hour', fontsize = fontsize)
plt.yticks(fontsize = fontsize - 2);

ax.get_legend().set_visible(False);

In [ ]:
# Total jobs ended each month
weekly_jobs_ended_by_hour = jobs.groupby(pd.Grouper(key='END',freq='W')).count()/168
weekly_jobs_ended_by_hour = weekly_jobs_ended_by_hour.reset_index()

# Total jobs completed each month
weekly_jobs_completed_by_hour = jobs[jobs['STATE'] == 'COMPLETED'].groupby(pd.Grouper(key='END',freq='M')).count()/168
weekly_jobs_completed_by_hour = weekly_jobs_completed_by_hour.reset_index()

# Format date ticks for x axis and define fontsize
date_labels = weekly_jobs_ended_by_hour['END'].dt.strftime('%Y-%m')
fontsize = 20

# Set figure size and graph plots
fig, ax = plt.subplots(figsize=(30,16))
sns.barplot(x = 'END', 
            y = 'JOBID', 
            data = weekly_jobs_ended_by_hour, 
            hue = date_labels, 
            dodge = False, 
            palette = "deep",
            ax = ax,
            alpha = 0.7)
# sns.barplot(x = 'END', 
#             y = 'JOBID', 
#             data = weekly_jobs_completed_by_hour, 
#             hue = date_labels, 
#             dodge = False, 
#             palette = "deep",
#             ax = ax)

plt.xlabel('Week', fontsize = fontsize)
plt.xticks(fontsize = fontsize - 2, rotation = 45)
ax.set_xticklabels(labels = date_labels)

plt.ylabel('Jobs Per Hour', fontsize = fontsize)
plt.yticks(fontsize = fontsize - 2);

ax.get_legend().set_visible(False);

In [ ]:
# Total jobs ended each month
weekly_jobs_ended_by_hour = jobs.groupby(pd.Grouper(key='END',freq='M')).count()/168
weekly_jobs_ended_by_hour = weekly_jobs_ended_by_hour.reset_index()

# Total jobs completed each month
weekly_jobs_completed_by_hour = jobs[jobs['STATE'] == 'COMPLETED'].groupby(pd.Grouper(key='END',freq='M')).count()/168
weekly_jobs_completed_by_hour = weekly_jobs_completed_by_hour.reset_index()

# Format date ticks for x axis and define fontsize
date_labels = weekly_jobs_ended_by_hour['END'].dt.strftime('%Y-%m')
fontsize = 20

# Set figure size and graph plots
fig, ax = plt.subplots(figsize=(30,16))
sns.barplot(x = 'END', 
            y = 'JOBID', 
            data = weekly_jobs_ended_by_hour, 
            hue = date_labels, 
            dodge = False, 
            palette = "deep",
            ax = ax,
            alpha = 0.7)
# sns.barplot(x = 'END', 
#             y = 'JOBID', 
#             data = weekly_jobs_completed_by_hour, 
#             hue = date_labels, 
#             dodge = False, 
#             palette = "deep",
#             ax = ax)

plt.xlabel('Week', fontsize = fontsize)
plt.xticks(fontsize = fontsize - 2)
ax.set_xticklabels(labels = date_labels)

plt.ylabel('Jobs Per Hour', fontsize = fontsize)
plt.yticks(fontsize = fontsize - 2);

ax.get_legend().set_visible(False);

## Question 2: What does the completions per hour look like over the time span of the dataset?

## Question 3: Are there weekly trends, and has it been increasing over the last year?

In [ ]:
jobs.groupby([jobs.END.dt.year, jobs.END.dt.month, jobs.END.dt.day, jobs.END.dt.hour]).count()

Boxplot = Group by month and graph ? on y-axis
Violin plot with x-axis = months (Oct. 2020 thru Oct. 2021) and y-axis = ?

It seems that the amount of jobs per hour increase in the August to September time frame. Coincidentally this correlates to the beginning of the school year. If we could analyze the data for the rest of 2020, 2019 and 2018, we could test further validate this hypothesis.

In [ ]:
daily_jobs_ended_per_hour = jobs.groupby(jobs['END'].dt.day_name()).count()['JOBID']/168
daily_jobs_ended_per_hour = daily_jobs_ended_per_hour.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

fig, ax = plt.subplots(figsize=(30,16))
daily_jobs_ended_per_hour.plot(kind='bar')

plt.xlabel('Day of the Week', fontsize=20)
day_labels = daily_jobs_ended_per_hour.index
ax.set_xticklabels(labels=day_labels, rotation=90, fontsize=20)

plt.ylabel('Jobs Per Hour', fontsize=20)
plt.yticks(fontsize=20);

On average, it looks like the completed jobs per hour are higher at the beginning of the week (Monday thru Wednesday) and decrease as the week goes on.

## Question 4: Does the job state affect completions per hour? i.e. if I only look at jobs with exit code 0:0 in the "COMPLETED" state, is that a similar number of completions per hour as with all jobs, failed or cancelled? This will indicate if the load on the scheduler is by user design or is a result of users not sufficiently testing their jobs before submitting very large arrays. We also expect that most job completions will be in the "production" partition, but is this actually true?

## Question 5: Calculate some descriptive statistics about how often the scheduler was unresponsive, how long these periods of time were, and create a time series plot of when the scheduler was having difficulties.

In [ ]:
logs['TIMELAPS'].str.extract('(\w+\s\d)')

In [ ]:
logs

In [ ]:
logs[(logs['USER'] == 'user 9204') 
     & (logs['RETURNCODE'] == 'returncode 1')
    & ()]

## Question 6: Finally, combine the time series information from the two datasets together to see how well correlated heavy job-completion load is with the unresponsiveness, and to see if there is some threshold of job completions per hour that generally results in unresponsiveness.